In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

Ref:
- https://towardsdatascience.com/building-mobilenet-from-scratch-using-tensorflow-ad009c5dd42c

In [3]:
def mobilenet_block(x, filters, strides):
    x = layers.DepthwiseConv2D(kernel_size=(3, 3), strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters=filters, kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

![](https://miro.medium.com/max/1302/1*l31pZKpoQmkiKHmQs4OZlA.png)

In [4]:
def MobileNet(num_classes):
    inputs = layers.Input(shape=(224, 224, 3))
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = mobilenet_block(x, 64, (1, 1))
    x = mobilenet_block(x, 128, (2, 2))
    x = mobilenet_block(x, 128, (1, 1))
    x = mobilenet_block(x, 256, (2, 2))
    x = mobilenet_block(x, 256, (1, 1))
    x = mobilenet_block(x, 512, (2, 2))
    for i in range(5):
        x = mobilenet_block(x, 512, (1, 1))
    
    x = mobilenet_block(x, 1024, (2, 2))
    x = mobilenet_block(x, 1024, (1, 1))
    x = layers.AveragePooling2D(pool_size=(7, 7), strides=(1,1), data_format='channels_first')(x)
    outputs = layers.Dense(units=num_classes, activation='softmax')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [5]:
model = MobileNet(num_classes=2)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 32)     128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 112, 112, 32)     288       
 Conv2D)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 32)     128   

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dataset

In [7]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory('D:\AI-Project\Pancarona PPSMB UGM - Gender', validation_split=0.2, subset='training', seed=1337, image_size=(224,224), batch_size=64, label_mode='int')
val_ds = tf.keras.preprocessing.image_dataset_from_directory('D:\AI-Project\Pancarona PPSMB UGM - Gender', validation_split=0.2, subset='validation', seed=1337, image_size=(224,224), batch_size=64, label_mode='int')

Found 9507 files belonging to 2 classes.
Using 7606 files for training.
Found 9507 files belonging to 2 classes.
Using 1901 files for validation.


In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import os





timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
name = 'mobilenet-'+timestr

checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.system('mkdir {}'.format(checkpoint_dir))

# save model after each epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1
)
tensorboard_callback = TensorBoard(
    log_dir='tensorboard_logs/'+name,
    histogram_freq=1
)

model.fit(
    train_ds,
    epochs=20,
    verbose=1,
    validation_data= val_ds,
    batch_size=64,
    callbacks=[cp_callback, tensorboard_callback]
)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "D:\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "D:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "D:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "D:\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "D:\Anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    F